In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import openpyxl
import pickle

In [2]:
#엑셀파일 생성
wb = openpyxl.Workbook("올리브영_크롤링.xlsx")        
ws = wb.create_sheet("시트명")             
ws.append(['브랜드','상품명','카테고리','정가','할인가','아이디','별점','피부정보','피부타입','피부고민','자극도'])  #컬럼명 제공


In [3]:
######################제공하는 코드 건드리지 마세요 #############################

#크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

# 브라우저 꺼짐 방지
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('detach', True)
User_Agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
chrome_options.add_argument(f"user-agent={User_Agent}")

# 불필요한 에러 메세지 없애기
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

# 크롬 드라이버 경로 지정
#driver_path = "/Applications/chromedriver-mac-arm64/chromedriver.exe"  # 여기에 자신이 설치한 크롬 드라이버의 경로를 입력

# 크롬 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

Error getting version of chromedriver 115. Retrying with chromedriver 114 (attempt 1/5)


In [ ]:
main_url ='https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010001&pageIdx=1&rowsPerPage=8'

driver.get(main_url)

with open('my_df.pkl', 'rb') as pkl:
    sub_url = pickle.load(pkl)
    
time.sleep(1)
# 1위 인덱스 0,  2위 인덱스1, 3위 인덱스2, 4위 인덱스3, 5위 인덱스4

In [ ]:
# 리뷰 한페이지당 고객 10명의 리뷰가 담겨 있음
# 고객 10명에 대한 리뷰 정보를 받아오는 함수
def customer_info():

    for j in range(0,10,1):

        #브랜드
        try:
            brand = driver.find_element(By.CSS_SELECTOR,'#moveBrandShop').text
        except:
            brand ="없음"

        #상품명
        try:
            p_name = driver.find_element(By.CSS_SELECTOR,'#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name').text
        except:
            p_name="없음"

        #카테고리
        try:
            p_category = driver.find_element(By.CSS_SELECTOR,'#dtlCatNm').text
        except:
            p_category="없음"

        #정가
        try:
            price = driver.find_element(By.CSS_SELECTOR,'#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike').text
        except:
            price=0

        #할인가
        try:
            discount = driver.find_element(By.CSS_SELECTOR,'#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong').text
        except:
            discount=0

        #고객 아이디
        try:
            _id = driver.find_element(By.CSS_SELECTOR,f'#gdasList > li:nth-child({j+1}) > div.info > div > p.info_user > a.id').text
        except:
            _id ="없음"

        #별점
        try:
            _star = driver.find_element(By.CSS_SELECTOR,f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.score_area > span.review_point > span').text
        except:
            _star="없음"

        #고객 피부 정보
        try:
            _info = list()
            information = driver.find_elements(By.CSS_SELECTOR,f'#gdasList > li:nth-child({j+1}) > div.info > div > p.tag>span')
            for info in information:
                _info.append(info.text)
            _info = ' '.join(_info)
            # span으로 이루어진 피부정보 다 가져오기
        except:
            _info = "없음"

        #피부 타입
        try:
            skin_type =  driver.find_element(By.CSS_SELECTOR,f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span ').text
        except:
            skin_type = "없음"
            
        #피부 고민
        try:
            skin_trouble = driver.find_element(By.CSS_SELECTOR,f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
        except:
            skin_trouble = "없음"
        #자극도
        try:
            skin_irritation = driver.find_element(By.CSS_SELECTOR,f'#gdasList > li:nth-child({j+1}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text
        except:
            skin_irritation = "없음"
            
        #엑셀 데이터 쌓기
        ws.append([brand,p_name,p_category,price,discount,_id, _star, _info, skin_type, skin_trouble, skin_irritation])
        time.sleep(1)

In [ ]:
#웹페이지 해당 주소 이동

for i in range(0,100):          # 여기 부분을 각자 맡은 랭킹에 맞춰서 조절하면 될거 같아요!! 예를 들어 1위부터 25위 까지 한다하면 for i in range(0,25): , 만약 51위부터 75위 까지라면 for i in range(50,75): 이렇게 변경하면 될 거 같아요!
    driver.implicitly_wait(5)  #웹페이지 로딩 될때까지 5초는 기다림
    driver.maximize_window()   #화면 최대화
    driver.get(sub_url[i])          
    time.sleep(3)

    #리뷰버튼 클릭
    rv = driver.find_element(By.CSS_SELECTOR,'#reviewInfo > a')
    rv.click()
    time.sleep(3)

    #리뷰 하단 끝까지 스크롤하는 함수 (빈칸없음.그대로 코드 사용가능)
    before_h = driver.execute_script("return window.scrollY")         #스크롤 전 높이
    #화면 맨아래까지 스크롤
    while True:
        driver.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
        time.sleep(1)

        #스크롤 후 높이
        after_h = driver.execute_script("return window.scrollY")

        #스크롤 값이 같으면 스크롤 멈춤
        if after_h == before_h:
            break
        before_h = after_h   

    last_page=False
    
    for k in range(1,101):  #100페이지 크롤링 한다고 했을 때 (상품당 최대 100페이지까지 있음)

        #마지막 페이지면 종료
        if last_page == True:
            break
        
        #페이지 숫자 10이하 일 때
        if k<11:
            try:    
                customer_info()
                if k != 10:
                   
                    next_page = driver.find_element(By.CSS_SELECTOR,f'#gdasContentsArea > div > div.pageing > a:nth-child({k+1})') #1번째 페이지면 2번째 페이지를 클릭해야하기 때문에
                    next_page.click()
                elif k == 10:          #10페이지 크롤링 한 후에 다음페이지로 가는 화살표 버튼 클릭

                    next_page = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.pageing > a.next')
                    next_page.click()
            except:
                last_page=True

       #페이지 숫자 11이상 일 때  (규칙을 찾아 각 페이지 크롤링 후 다음 페이지로 이동하도록 코드 작성)        
        else :
            # 10으로 나눈뒤 나머지 + 1을 하면 다음페이지 번호가 됨
            k_mod = (k%10) # k를 10으로 나눈 나머지
            try:
                customer_info()
                if k_mod != 0:
                    next_page = driver.find_element(By.CSS_SELECTOR,f'#gdasContentsArea > div > div.pageing > a:nth-child({k_mod+2})') # 이전과는 다르게 2를 더해줘야함 -> 이전으로 가는 페이지가 a로 포함되기 때문!
                    # gdasContentsArea > div > div.pageing > a:nth-child(3) 12페이지로 가는 버튼 link
                    next_page.click()
                else:          #10페이지 크롤링 한 후에 다음페이지로 가는 화살표 버튼 클릭
                    next_page = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.pageing > a.next')
                    next_page.click()
            except:
                last_page=True
                   

driver.quit()

In [ ]:
wb.save("올리브영_크롤링(31_50).xlsx")       